In [ ]:
import requests
import pandas as pd
import datetime
import time
from openpyxl import Workbook
from openpyxl.workbook.protection import WorkbookProtection

# Define the Alpha Vantage API and API key
api_key = 'AM0J3JHW35CBI4KU'
base_url = 'https://www.alphavantage.co/query?'
symbols = ['GS', 'JPM', 'MS']

# Define the API functions to retrieve data
functions = ['INCOME_STATEMENT', 'BALANCE_SHEET', 'CASH_FLOW']

# Get the current date
current_date = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# Create a new Excel writer object using the openpyxl package to write to a '.xlsx' file
# ('.xls' format will be removed in a future version of pandas.)
workbook = Workbook()

# Set a password for the workbook
password = 'Team_17_SVB'
workbook.security = WorkbookProtection(workbookPassword=password)

writer = pd.ExcelWriter(current_date + '_av_company_financial_data' + '.xlsx', engine='openpyxl')
writer.book = workbook

# Iterate over the symbols
for symbol in symbols:
    # Iterate over the functions
    for function in functions:
        # Define the API parameters
        params = {'function': function, 'symbol': symbol, 'apikey': api_key}

        # Make the API request
        response = requests.get(base_url, params=params)

        # Get the JSON data from the response
        json_data = response.json()

        # Extract the relevant data from the JSON response
        if 'quarterlyReports' in json_data:
            data = json_data['quarterlyReports']
        else:
            data = []

        # Create a DataFrame from the extracted data
        df = pd.DataFrame(data)

        # Write the DataFrame to a new sheet in the Excel file
        sheet_name = f'{symbol}_{function}'
        df.to_excel(writer, sheet_name=sheet_name, index=False)

        # Delay for 12 seconds to comply with the API limitation
        time.sleep(12)

# Save the Excel file
writer.save()

# Print a message after saving the data
print('Data saved to ' + current_date + '_av_company_financial_data' + '.xlsx')

# Create a new Excel writer object
workbook_filtered = Workbook()

# Set a password for the workbook
workbook_filtered.security = WorkbookProtection(workbookPassword=password)

writer_filtered = pd.ExcelWriter(current_date + '_filtered_company_financial_data' + '.xlsx', engine='openpyxl')
writer_filtered.book = workbook_filtered

for sheet_name in writer.sheets:
    # Convert the data Series to a DataFrame
    df = pd.read_excel(writer, sheet_name=sheet_name)

    # Filter the columns based on the modified column names
    relevant_columns = [
        'fiscaldateending',
        'commonstockequity',
        'netdebt',
        'ordinarysharesnumber',
        'preferredstockequity',
        'tangiblebookvalue',
        'totalcapitalization',
        'totalequitygrossminorityinterest',
        'treasurysharesnumber',
        'investedcapital',
        'nettangibleassets',
        'preferredsharesnumber',
        'shareissued',
        'totalassets',
        'totaldebt',
        'totalliabilitiesnetminorityinterest',
        'totallassests',
        'totalliabilities',
        'shareholdersequity'
    ]

    # Modify column names for case insensitivity and spaces
    df.columns = [col.replace(' ', '').lower() for col in df.columns]

    # Filter the DataFrame based on the relevant columns
    filtered_data = df.reindex(columns=relevant_columns)

    # Save the filtered DataFrame to a new sheet in the Excel file
    filtered_data.to_excel(writer_filtered, sheet_name=sheet_name, index=False)

# Save the Excel file
writer_filtered.save()

# Print a message after saving the data
print('Filtered data saved to ' + current_date + '_filtered_company_financial_data.xlsx')
